Main objective of this recommendation is to encourage the outdoor trends of dining and the sentiment towards particular cuisines,
Ratings


#Importing Libraries



In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package stopwords to /Users/bunty/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
data_names = pd.read_csv('/Users/bunty/Code/MachineLearning/Restaurant_Recommendation/Restaurant names and Metadata.csv')
data_review = pd.read_csv('/Users/bunty/Code/MachineLearning/Restaurant_Recommendation/Restaurant reviews.csv')

In [48]:
print(data_names.shape)
data_names.sample(5)

(105, 6)


,Name,Links,Cost,Collections,Cuisines,Timings
99,Chinese Pavilion,https://www.zomato.com/hyderabad/chinese-pavil...,"1,000",NaN,"Chinese, Seafood","12 Noon to 3:30 PM, 7 PM to 11 PM"
97,Zega - Sheraton Hyderabad Hotel,https://www.zomato.com/hyderabad/zega-sheraton...,"1,750",NaN,"Asian, Sushi",12Noon to 2AM (Mon-Sun)
17,Hotel Zara Hi-Fi,https://www.zomato.com/hyderabad/hotel-zara-ga...,400,NaN,"Chinese, North Indian",11:30 AM to 1 AM
81,Delhi-39,https://www.zomato.com/hyderabad/delhi-39-gach...,600,NaN,"North Indian, Chinese",11am to 11pm (Mon-Sun)
88,Domino's Pizza,https://www.zomato.com/hyderabad/dominos-pizza...,400,NaN,"Fast Food, Pizza",11 AM to 11 PM


In [49]:
print(data_review.shape)
data_review.sample(5)

(10000, 7)


,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures
8909,Mohammedia Shawarma,Tanveer Mansoori,This is my n-th experience ordering the Shawar...,5,1 Review,2/9/2019 0:00,0
1646,NorFest - The Dhaba,G. Sushmitha Reddy,They definitely promise you a decent meal.\n\n...,3,"81 Reviews , 872 Followers",10/11/2018 15:10,4
4000,Frio Bistro,Oindrila Dutta,One of my favorite restaurants in the locality...,5,"6 Reviews , 2 Followers",5/17/2019 20:01,0
3752,Kritunga Restaurant,Madhu Choppara,delivery good,5,2 Reviews,8/2/2018 21:01,0
1805,10 Downing Street,Rmandava70,Place is awesome so their craft beers. Liked B...,4,"92 Reviews , 727 Followers",5/9/2019 18:56,4


In [50]:
data_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         105 non-null    object
 1   Links        105 non-null    object
 2   Cost         105 non-null    object
 3   Collections  51 non-null     object
 4   Cuisines     105 non-null    object
 5   Timings      104 non-null    object
dtypes: object(6)
memory usage: 5.0+ KB


In [51]:
data_names.nunique()

Name           105
Links          105
Cost            29
Collections     42
Cuisines        92
Timings         77
dtype: int64

In [52]:
data_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Restaurant  10000 non-null  object
 1   Reviewer    9962 non-null   object
 2   Review      9955 non-null   object
 3   Rating      9962 non-null   object
 4   Metadata    9962 non-null   object
 5   Time        9962 non-null   object
 6   Pictures    10000 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [53]:
data_review.nunique()

Restaurant     100
Reviewer      7446
Review        9364
Rating          10
Metadata      2477
Time          9782
Pictures        36
dtype: int64

In [54]:
# rename restaurant to name to merge into single dataset
data_review = data_review.rename(columns={'Restaurant':'Name'})
data_review.head()

,Name,Reviewer,Review,Rating,Metadata,Time,Pictures
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0


In [55]:
# merging two dataset
df = pd.merge(data_names, data_review, how='left', on ='Name')
df.head(3)
# df.to_csv("Merge.csv")

,Name,Links,Cost,Collections,Cuisines,Timings,Reviewer,Review,Rating,Metadata,Time,Pictures
0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0.0
1,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0.0
2,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0.0


Data Processing with Merge data 

In [56]:
merge_df = pd.read_csv('/Users/bunty/Code/MachineLearning/Restaurant_Recommendation/Merge_data.csv')

In [76]:
merge_df.head(5)

,Unnamed: 0,Name,Links,Cost,Collections,Cuisines,Timings,Reviewer,Review,Rating,Metadata,Time,Pictures
0,0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0.0
1,1,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0.0
2,2,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0.0
3,3,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0.0
4,4,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0.0


In [80]:
required_columns = ['Cuisines', 'Rating', 'Cost', 'Timings']

# Load only the required columns from the CSV file
new_merge_df = pd.read_csv('/Users/bunty/Code/MachineLearning/Restaurant_Recommendation/Merge_data.csv', usecols=required_columns)

# Specify the desired column order
desired_column_order = ['Cuisines', 'Rating', 'Cost', 'Timings']

# Reorder the columns
new_merge_df = new_merge_df[desired_column_order]

# Now, 'df' contains only the specified columns
print(new_merge_df.head())

                                            Cuisines Rating Cost  \
0  Chinese, Continental, Kebab, European, South I...      5  800   
1  Chinese, Continental, Kebab, European, South I...      5  800   
2  Chinese, Continental, Kebab, European, South I...      5  800   
3  Chinese, Continental, Kebab, European, South I...      5  800   
4  Chinese, Continental, Kebab, European, South I...      5  800   

                                         Timings  
0  12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)  
1  12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)  
2  12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)  
3  12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)  
4  12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)  


In [81]:
new_merge_df.to_csv("NewMerge.csv")

# Data Cleaning

In [57]:
df.drop(['Reviewer','Time','Pictures','Links','Collections'], axis =1, inplace=True)
df.head(3)

,Name,Cost,Cuisines,Timings,Review,Rating,Metadata
0,Beyond Flavours,800,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)","The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers"
1,Beyond Flavours,800,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers"
2,Beyond Flavours,800,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers"


In [58]:
# changing cost and rating column
df['Cost'] = df['Cost'].apply(str).str.replace(',', '').astype(float)
df['Rating'] = df['Rating'].str.replace('Like','1').astype(float)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10005 entries, 0 to 10004
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      10005 non-null  object 
 1   Cost      10005 non-null  float64
 2   Cuisines  10005 non-null  object 
 3   Timings   9905 non-null   object 
 4   Review    9955 non-null   object 
 5   Rating    9962 non-null   float64
 6   Metadata  9962 non-null   object 
dtypes: float64(2), object(5)
memory usage: 625.3+ KB


,Name,Cost,Cuisines,Timings,Review,Rating,Metadata
0,Beyond Flavours,800.0,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)","The ambience was good, food was quite good . h...",5.0,"1 Review , 2 Followers"
1,Beyond Flavours,800.0,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Ambience is too good for a pleasant evening. S...,5.0,"3 Reviews , 2 Followers"
2,Beyond Flavours,800.0,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",A must try.. great food great ambience. Thnx f...,5.0,"2 Reviews , 3 Followers"
3,Beyond Flavours,800.0,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Soumen das and Arun was a great guy. Only beca...,5.0,"1 Review , 1 Follower"
4,Beyond Flavours,800.0,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",Food is good.we ordered Kodi drumsticks and ba...,5.0,"3 Reviews , 2 Followers"


In [59]:
print('No of data inputs',len(df))
print('no of NaN values for each column:\n')
print(df.isnull().sum())

No of data inputs 10005
no of NaN values for each column:

Name          0
Cost          0
Cuisines      0
Timings     100
Review       50
Rating       43
Metadata     43
dtype: int64


In [60]:
df['Name'][df['Rating'].isnull()== True].value_counts()

American Wild Wings                       23
Arena Eleven                              15
IndiBlaze                                  1
Angaara Counts 3                           1
Wich Please                                1
Republic Of Noodles - Lemon Tree Hotel     1
Sweet Basket                               1
Name: Name, dtype: int64

In [61]:
print('Mean of Rating for American Wild Wings: ',df['Rating'][df['Name'] == 'American Wild Wings'].mean())
print('Mean of Rating for Arena Eleven: ',df['Rating'][df['Name'] == 'Arena Eleven'].mean())

print('Overall. Mean of ratings: ',df['Rating'].mean())


Mean of Rating for American Wild Wings:  3.9740259740259742
Mean of Rating for Arena Eleven:  4.117647058823529
Overall. Mean of ratings:  3.6007829753061635


In [62]:
df['Rating'].fillna(4,inplace = True)
df['Review'] = df['Review'].fillna('-')
# changing nan review by - becoz it will not create any problem

df.isnull().sum()

Name          0
Cost          0
Cuisines      0
Timings     100
Review        0
Rating        0
Metadata     43
dtype: int64

# Text Preprocessing and Cleaning

Using Review and Cuisines feature inorder to create recommender system

In [63]:
# df[['Review','Cuisines']].head(5)
# df.sample(5)

In [64]:
stemmer = PorterStemmer()

In [65]:
#  stemming
def stem_word(text):
    txt = " ".join([stemmer.stem(w) for w in text.split()])
    return txt

In [66]:
df['Review'] = df['Review'].map(lambda x: stem_word(x))
df['Cuisines'] = df['Cuisines'].map(lambda x:stem_word(x))

In [67]:
df[['Review','Cuisines']].head(5)

,Review,Cuisines
0,"the ambienc wa good, food wa quit good . had s...","chinese, continental, kebab, european, south i..."
1,ambienc is too good for a pleasant evening. se...,"chinese, continental, kebab, european, south i..."
2,a must try.. great food great ambience. thnx f...,"chinese, continental, kebab, european, south i..."
3,soumen da and arun wa a great guy. onli becaus...,"chinese, continental, kebab, european, south i..."
4,food is good.w order kodi drumstick and basket...,"chinese, continental, kebab, european, south i..."


In [68]:
# df[['Name','Review','Cuisines']].head(5)

df.sample()

,Name,Cost,Cuisines,Timings,Review,Rating,Metadata
8982,Mohammedia Shawarma,150.0,"street food, arabian",1 PM to 1 AM,horribl shawarma,1.0,"2 Reviews , 45 Followers"


In [69]:
# Changing data set index by restaurant name
df.set_index('Name', inplace=True)

# Saving indexes in a series
indices = pd.Series(df.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Review'])

# Calculating cosine similarities
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [70]:
def recommend(name, cosine_similarities = cosine_similarities):

    # Create a list to put top 10 restaurants
    recommend_restaurant = []

    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]

    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)

    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)

    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df.index)[each])

    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['Cuisines', 'Rating', 'Cost', 'Timings'])

    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df[['Cuisines','Rating', 'Cost', 'Timings']][df.index == each].sample()))

    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['Cuisines','Rating', 'Cost'], keep=False)
    df_new = df_new.sort_values(by='Rating', ascending=False).head(7)

    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))

    return df_new

    # if we say we have to recommend on the basis of food
    #  the best north indian restaurant woul be visible by searching north indian

In [71]:
# HERE IS A RANDOM RESTAURANT. LET'S SEE THE DETAILS ABOUT THIS RESTAURANT:
df[df.index == 'Karachi Bakery'].head(1)

,Cost,Cuisines,Timings,Review,Rating,Metadata
Name,,,,,,
Karachi Bakery,500.0,"bakery, fast food, dessert",10 AM to 10 PM,well you know what they are famou for - karach...,5.0,"207 Reviews , 400 Followers"


In [72]:
# LET'S SEE WHAT ARE WE GOING TO BE RECOMMENDED:
recommend('Labonel')

TOP 7 RESTAURANTS LIKE Labonel WITH SIMILAR REVIEWS: 


,Cuisines,Rating,Cost,Timings
The Old Madras Baking Company,bakeri,5.0,350.0,7 AM to 11 PM
Shree Santosh Dhaba Family Restaurant,"north indian, chinese, fast food",5.0,500.0,11 AM to 11:30 PM
Karachi Cafe,"cafe, desserts, fast food",5.0,600.0,10 AM to 10 PM
The Chocolate Room,"cafe, dessert",4.0,600.0,10 AM to 12 Midnight
Labonel,"bakery, dessert",2.0,1000.0,"11 AM to 8 PM (Mon-Sat), Sun Closed"
The Chocolate Room,"cafe, dessert",2.0,600.0,10 AM to 12 Midnight
Karachi Bakery,"bakery, fast food, dessert",1.0,500.0,10 AM to 10 PM


Saving the Model

In [73]:
import pickle 

In [74]:
filename = 'trained_model.sav'

In [75]:
# Saving the model
with open('trained_model.sav', 'wb') as model_file:
    pickle.dump(cosine_similarities, model_file)